# Dados da Vacinação de Sergipe por Município

Neste notebook você encontra todos os scripts para processamento dos boletins da vacinação publicados quase que diariamente pela secretaria de estado da saúde. 

Os boletins são publicados em formato excel e baixados manualmente para a pasta input. Os boletins forma renomeados para o formato **YYYY-MM-DD.xlsx**. 

## Processando os dados

In [1]:
import pandas as pd

A partir dos boletins publicados vai ser gerada uma base única que informe para cada dia, o total de doses aplicadas em cada um dos municípios de Sergipe. Para cada arquivo, as colunas de interesse serão a do nome do município e a do total de doses enviadas e aplicadas. No entanto, essas colunas de interesse vão mudando nos arquivos a medida que a vacinação vai avançando pela inclusão de novos tipos de doses ou mudança na estrutura do arquivo. Sendo assim, essas colunas de interesse vão mudar a depender do arquivo que esteja sendo processando. 

Foram definidos seis tipos de arquivos. Cada tipo corresponde a um conjunto de arquivos em um intervalo de tempo. Essa verificação foi feita manualmente olhando cada um dos arquivos para perceber a mudança na estrutura. Os tipos e intervalos de datas são: 


* **Tipo 1:** de 03/02/21 a 08/02/21
* **Tipo 2:** de 09/02/21 a 08/03/21
* **Tipo 3:** de 11/03/21 a 27/06/21
* **Tipo 4:** de 28/06/21 a 06/07/21
* **Tipo 5:** de 07/07/21 a 23/09/21
* **Tipo 6:** de 24/09/21 em diante.

Para cada tipo foi definida uma função de processamento para tratar cada particularidade.

Além da seleção das colunas de interesse, foi necessário fazer uma padronização dos códigos e nomes dos municípios, principalemente nos primeiros arquivos. Para isso vamos utilizar a tabela de diretórios brasileiros do site [Base dos Dados](https://basedosdados.org/dataset/br-bd-diretorios-brasil). Os dados foram extraídos a partir da consulta SQL:

```sql
SELECT * FROM `basedosdados.br_bd_diretorios_brasil.municipio` where sigla_uf = 'SE' 
```

Os resultados foram salvos no arquivo .csv `diretorio_brasileiro_se.csv` na pasta `input`.

In [2]:
data_diretorio = pd.read_csv("../input/diretorio_brasileiro_se.csv")
data_diretorio

,id_municipio,id_municipio_6,id_municipio_tse,id_municipio_rf,id_municipio_bcb,nome,capital_uf,id_comarca,id_regiao_saude,regiao_saude,...,regiao_intermediaria,id_microrregiao,microrregiao,id_mesorregiao,mesorregiao,ddd,id_uf,sigla_uf,nome_uf,regiao
0,2800100,280010,31011,3101,25690.0,Amparo de São Francisco,0,2801603,28007,Propriá,...,Aracaju,28007,Propriá,2803,Leste Sergipano,79,28,SE,Sergipe,Nordeste
1,2800209,280020,31038,3103,40008.0,Aquidabã,0,2800209,28007,Propriá,...,Aracaju,28003,Nossa Senhora das Dores,2802,Agreste Sergipano,79,28,SE,Sergipe,Nordeste
2,2800308,280030,31054,3105,13530.0,Aracaju,1,2800308,28001,Aracaju,...,Aracaju,28011,Aracaju,2803,Leste Sergipano,79,28,SE,Sergipe,Nordeste
3,2800407,280040,31070,3107,26486.0,Arauá,0,2800407,28002,Estância,...,Aracaju,28012,Boquim,2803,Leste Sergipano,79,28,SE,Sergipe,Nordeste
4,2800506,280050,31097,3109,30960.0,Areia Branca,0,2803609,28003,Itabaiana,...,Itabaiana,28004,Agreste de Itabaiana,2802,Agreste Sergipano,79,28,SE,Sergipe,Nordeste
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,2807204,280720,32433,3243,5971.0,Siriri,0,2804607,28006,Nossa Senhora do Socorro,...,Aracaju,28008,Cotinguiba,2803,Leste Sergipano,79,28,SE,Sergipe,Nordeste
71,2807303,280730,32450,3245,11745.0,Telha,0,2801603,28007,Propriá,...,Aracaju,28007,Propriá,2803,Leste Sergipano,79,28,SE,Sergipe,Nordeste
72,2807402,280740,32476,3247,17543.0,Tobias Barreto,0,2807402,28004,Lagarto,...,Itabaiana,28005,Tobias Barreto,2802,Agreste Sergipano,79,28,SE,Sergipe,Nordeste
73,2807501,280750,32492,3249,21931.0,Tomar do Geru,0,2801702,28002,Estância,...,Aracaju,28012,Boquim,2803,Leste Sergipano,79,28,SE,Sergipe,Nordeste


In [3]:
# Data limite para leitura dos boletins de vacinação.
data_limite = '2022-02-15'

Os problemas encontraddos em relação aos nomes foram a falta de código de município ou o nome escrito em caixa alta e sem acento. Para tratar isto foram criadas duas funções auxiliares que retorna o código dado o nome e o nome dado o código.

### Funções Auxiliares

Atualiza um campo do data frame com uma informação baseada em outra coluna do dataframe.

In [4]:
def change_codigo_6_7(row):
    codigo_municipio = row['codigo_municipio']
    nome_change = data_diretorio.loc[data_diretorio['id_municipio_6'] == int(codigo_municipio)]['id_municipio']

    return nome_change.item()

In [5]:
def change_name(row):

    codigo_municipio = row['codigo_municipio']
    nome_change = data_diretorio.loc[data_diretorio['id_municipio_6'] == int(codigo_municipio)]['nome']
   
    return nome_change.item()

In [6]:
def change_codigo(row):
    nome_municipio = row['nome_municipio']
    codigo_change = data_diretorio.loc[data_diretorio['nome'] == nome_municipio]['id_municipio']

    return codigo_change.item()

### Funções de Processamento

In [7]:
def process_type1(file, output): 

    file_name = file.split("/")[-1].split(".")[0]
    order_columns = ['codigo_municipio','nome_municipio','enviadas_dose1', 'enviadas_dose2', 'enviadas_dose_unica', 'enviadas_dose_reforco','aplicadas_dose1','aplicadas_dose2','aplicadas_dose_unica','aplicadas_dose_reforco']

    data_ = pd.read_excel(file, skiprows=2,header=None)

    if file_name == '2021-02-05':
        data_ = data_.iloc[:, [0,-3]]
    else:
        data_ = data_.iloc[:, [0,-1]]



    data_ = data_.rename(columns={data_.columns[0]: 'nome_municipio', data_.columns[-1]: 'aplicadas_dose1'})
    data_ = data_.fillna(0)
    data_['nome_municipio'] = data_['nome_municipio'].str.replace("-", " ")
    data_['codigo_municipio'] = data_['nome_municipio'].str[0:6]
    data_['nome_municipio'] = data_['nome_municipio'].str[7:].str.strip()
    data_['aplicadas_dose1'] = data_['aplicadas_dose1'].round().astype(int)
    data_['aplicadas_dose2'] = 0
    data_['aplicadas_dose_unica'] = 0
    data_['aplicadas_dose_reforco'] = 0
    data_['enviadas_dose1'] = 0
    data_['enviadas_dose2'] = 0
    data_['enviadas_dose_unica'] = 0
    data_['enviadas_dose_reforco'] = 0
    data_ = data_.iloc[0:75, :]
    data_ = data_[order_columns]

    data_['nome_municipio'] = data_.apply(lambda row: change_name(row), axis=1)
    data_['codigo_municipio'] = data_.apply(lambda row: change_codigo_6_7(row), axis=1)

    data_.to_csv(output + "/" + file_name + ".csv", index=False) 
    
    

In [8]:
def process_type2(file, output): 

    file_name = file.split("/")[-1].split(".")[0]
    order_columns = ['codigo_municipio','nome_municipio','enviadas_dose1', 'enviadas_dose2', 'enviadas_dose_unica', 'enviadas_dose_reforco','aplicadas_dose1','aplicadas_dose2','aplicadas_dose_unica','aplicadas_dose_reforco']

    data_ = pd.read_excel(file, skiprows=9,header=None)
    data_ = data_.iloc[:, [0,-6,-5,-3,-2]]
    data_ = data_.rename(columns={
        data_.columns[0]: 'nome_municipio', 
        data_.columns[1]: 'enviadas_dose1', 
        data_.columns[2]: 'aplicadas_dose1',
        data_.columns[3]: 'enviadas_dose2',
        data_.columns[4]: 'aplicadas_dose2'
    })
    data_ = data_.fillna(0)
    data_['nome_municipio'] = data_['nome_municipio']
    data_['enviadas_dose1'] = data_['enviadas_dose1'].round().astype(int)
    data_['enviadas_dose2'] = data_['enviadas_dose2'].round().astype(int)
    data_['aplicadas_dose1'] = data_['aplicadas_dose1'].round().astype(int)
    data_['aplicadas_dose2'] = data_['aplicadas_dose2'].round().astype(int)
    data_['codigo_municipio'] = 0
    data_['aplicadas_dose_unica'] = 0
    data_['aplicadas_dose_reforco'] = 0
    data_['enviadas_dose_unica'] = 0
    data_['enviadas_dose_reforco'] = 0
    
    data_ = data_.iloc[0:75, :]
    data_ = data_[order_columns]

    data_['codigo_municipio'] = data_.apply(lambda row: change_codigo(row), axis=1)    

    data_.to_csv(output + "/" + file_name + ".csv", index=False)

In [9]:
def process_type3(file, output): 

    file_name = file.split("/")[-1].split(".")[0]
    order_columns = ['codigo_municipio','nome_municipio','enviadas_dose1', 'enviadas_dose2', 'enviadas_dose_unica', 'enviadas_dose_reforco','aplicadas_dose1','aplicadas_dose2','aplicadas_dose_unica','aplicadas_dose_reforco']

    data_ = pd.read_excel(file, skiprows=3,header=None)
    data_ = data_.iloc[:, [0,5,6,7,8]]
    data_ = data_.rename(columns={data_.columns[0]: 'nome_municipio', data_.columns[1]: 'enviadas_dose1', data_.columns[2]: 'enviadas_dose2', data_.columns[3]: 'aplicadas_dose1', data_.columns[4]: 'aplicadas_dose2'})
    data_ = data_.fillna(0)
    data_['codigo_municipio'] = data_['nome_municipio'].str[0:6]
    data_['nome_municipio'] = data_['nome_municipio'].str[7:]
    data_['nome_municipio'] = data_['nome_municipio'].str.strip()
    data_['enviadas_dose1'] = data_['enviadas_dose1'].round().astype(int)
    data_['enviadas_dose2'] = data_['enviadas_dose2'].round().astype(int)
    data_['aplicadas_dose1'] = data_['aplicadas_dose1'].round().astype(int)
    data_['aplicadas_dose2'] = data_['aplicadas_dose2'].round().astype(int)
    data_['enviadas_dose_unica'] = 0
    data_['enviadas_dose_reforco'] = 0
    data_['aplicadas_dose_unica'] = 0
    data_['aplicadas_dose_reforco'] = 0

    
    data_ = data_.iloc[0:75, :]
    data_ = data_[order_columns]

    data_['nome_municipio'] = data_.apply(lambda row: change_name(row), axis=1)
    data_['codigo_municipio'] = data_.apply(lambda row: change_codigo_6_7(row), axis=1)

    data_.to_csv(output + "/" + file_name + ".csv", index=False)

In [10]:
def process_type4(file, output): 

    file_name = file.split("/")[-1].split(".")[0]
    order_columns = ['codigo_municipio','nome_municipio','enviadas_dose1', 'enviadas_dose2', 'enviadas_dose_unica', 'enviadas_dose_reforco','aplicadas_dose1','aplicadas_dose2','aplicadas_dose_unica','aplicadas_dose_reforco']

    data_ = pd.read_excel(file, skiprows=3,header=None)
    data_ = data_.iloc[:, [0,5,6,7,8,-23]]
    data_ = data_.rename(columns={
        data_.columns[0]: 'nome_municipio', 
        data_.columns[1]: 'enviadas_dose1', 
        data_.columns[2]: 'enviadas_dose2', 
        data_.columns[3]: 'aplicadas_dose1', 
        data_.columns[4]: 'aplicadas_dose2', 
        data_.columns[5]: 'aplicadas_dose_unica'})
    data_ = data_.fillna(0)
    data_['codigo_municipio'] = data_['nome_municipio'].str[0:6]
    data_['nome_municipio'] = data_['nome_municipio'].str[7:]
    data_['nome_municipio'] = data_['nome_municipio'].str.strip()
    data_['enviadas_dose1'] = data_['enviadas_dose1'].round().astype(int)
    data_['enviadas_dose2'] = data_['enviadas_dose2'].round().astype(int)
    data_['aplicadas_dose1'] = data_['aplicadas_dose1'].round().astype(int)
    data_['aplicadas_dose2'] = data_['aplicadas_dose2'].round().astype(int)
    data_['aplicadas_dose_unica'] = data_['aplicadas_dose_unica'].round().astype(int)
    data_['aplicadas_dose_reforco'] = 0
    data_['enviadas_dose_reforco'] = 0
    data_['enviadas_dose_unica'] = 0

    
    
    data_ = data_.iloc[0:75, :]
    data_ = data_[order_columns]

    data_['nome_municipio'] = data_.apply(lambda row: change_name(row), axis=1)
    data_['codigo_municipio'] = data_.apply(lambda row: change_codigo_6_7(row), axis=1)

    data_.to_csv(output + "/" + file_name + ".csv", index=False)


In [11]:
def process_type5(file, output): 

    file_name = file.split("/")[-1].split(".")[0]
    order_columns = ['codigo_municipio','nome_municipio','enviadas_dose1', 'enviadas_dose2', 'enviadas_dose_unica', 'enviadas_dose_reforco','aplicadas_dose1','aplicadas_dose2','aplicadas_dose_unica','aplicadas_dose_reforco']

    data_ = pd.read_excel(file, skiprows=3,header=None)
    data_ = data_.iloc[:, [0,5,6,7,8,9,10]]
    data_ = data_.rename(columns={
        data_.columns[0]: 'nome_municipio', 
        data_.columns[1]: 'enviadas_dose1', 
        data_.columns[2]: 'enviadas_dose2',
        data_.columns[3]: 'enviadas_dose_unica', 
        data_.columns[4]: 'aplicadas_dose1', 
        data_.columns[5]: 'aplicadas_dose2', 
        data_.columns[6]: 'aplicadas_dose_unica'})
    data_ = data_.fillna(0)
    data_['codigo_municipio'] = data_['nome_municipio'].str[0:6]
    data_['nome_municipio'] = data_['nome_municipio'].str[7:]
    data_['nome_municipio'] = data_['nome_municipio'].str.strip()
    data_['enviadas_dose1'] = data_['enviadas_dose1'].round().astype(int)
    data_['enviadas_dose2'] = data_['enviadas_dose2'].round().astype(int)
    data_['enviadas_dose_unica'] = data_['enviadas_dose_unica'].round().astype(int)
    data_['aplicadas_dose1'] = data_['aplicadas_dose1'].round().astype(int)
    data_['aplicadas_dose2'] = data_['aplicadas_dose2'].round().astype(int)
    data_['aplicadas_dose_unica'] = data_['aplicadas_dose_unica'].round().astype(int)
    data_['aplicadas_dose_reforco'] = 0
    data_['enviadas_dose_reforco'] = 0
    
    
    
    data_ = data_.iloc[0:75, :]
    data_ = data_[order_columns]

    data_['nome_municipio'] = data_.apply(lambda row: change_name(row), axis=1)
    data_['codigo_municipio'] = data_.apply(lambda row: change_codigo_6_7(row), axis=1)

    data_.to_csv(output + "/" + file_name + ".csv", index=False)

In [12]:
def process_type6(file, output): 

    file_name = file.split("/")[-1].split(".")[0]
    order_columns = ['codigo_municipio','nome_municipio','enviadas_dose1', 'enviadas_dose2', 'enviadas_dose_unica', 'enviadas_dose_reforco','aplicadas_dose1','aplicadas_dose2','aplicadas_dose_unica','aplicadas_dose_reforco']

    data_ = pd.read_excel(file, skiprows=3,header=None)
    data_ = data_.iloc[:, [0,5,6,7,8,9,10,11,12]]
    data_ = data_.rename(columns={
        data_.columns[0]: 'nome_municipio',
        data_.columns[1]: 'enviadas_dose1', 
        data_.columns[2]: 'enviadas_dose2',
        data_.columns[3]: 'enviadas_dose_unica',  
        data_.columns[4]: 'enviadas_dose_reforco', 
        data_.columns[5]: 'aplicadas_dose1',
        data_.columns[6]: 'aplicadas_dose2', 
        data_.columns[7]: 'aplicadas_dose_unica', 
        data_.columns[8]: 'aplicadas_dose_reforco'})
    data_ = data_.fillna(0)
    data_['codigo_municipio'] = data_['nome_municipio'].str[0:6]
    data_['nome_municipio'] = data_['nome_municipio'].str[7:]
    data_['nome_municipio'] = data_['nome_municipio'].str.strip()
    data_['enviadas_dose1'] = data_['enviadas_dose1'].round().astype(int)
    data_['enviadas_dose2'] = data_['enviadas_dose2'].round().astype(int)
    data_['enviadas_dose_unica'] = data_['enviadas_dose_unica'].round().astype(int)
    data_['enviadas_dose_reforco'] = data_['enviadas_dose_reforco'].round().astype(int)
    data_['aplicadas_dose1'] = data_['aplicadas_dose1'].round().astype(int)
    data_['aplicadas_dose2'] = data_['aplicadas_dose2'].round().astype(int)
    data_['aplicadas_dose_unica'] = data_['aplicadas_dose_unica'].round().astype(int)
    data_['aplicadas_dose_reforco'] = data_['aplicadas_dose_reforco'].round().astype(int)
    
    
    
    data_ = data_.iloc[0:75, :]
    data_ = data_[order_columns]

    data_['nome_municipio'] = data_.apply(lambda row: change_name(row), axis=1)
    data_['codigo_municipio'] = data_.apply(lambda row: change_codigo_6_7(row), axis=1)

    data_.to_csv(output + "/" + file_name + ".csv", index=False)

In [13]:
def parse_date_list(list_):
    result_ = [datetime.strptime(x, '%Y-%m-%d') for x in list_]
    return result_

In [14]:
def get_type(date_):
    type1 = parse_date_list(['2021-02-03','2021-02-08'])
    type2 = parse_date_list(['2021-02-09','2021-03-08'])
    type3 = parse_date_list(['2021-03-11', '2021-06-27'])
    type4 = parse_date_list(['2021-06-28', '2021-07-06'])
    type5 = parse_date_list(['2021-07-07','2021-09-23'])
    type6 = parse_date_list(['2021-09-24',data_limite])
    
    if date_ >= type1[0] and date_ <= type1[1]:
        return 1
    elif date_ >= type2[0] and date_ <= type2[1]:
        return 2
    elif date_ >= type3[0] and date_ <= type3[1]:
        return 3
    elif date_ >= type4[0] and date_ <= type4[1]:
        return 4
    elif date_ >= type5[0] and date_ <= type5[1]:
        return 5
    elif date_ >= type6[0] and date_ <= type6[1]:
        return 6
    else:
        return -1

In [15]:
files_temp = os.listdir('../tmp/boletins_temporarios')
files_temp = [f.split(".")[0] for f in files_temp]

In [16]:
import os 
from datetime import datetime

path_ = "../input/boletins_vacinacao/"
path_temp = "../tmp/boletins_temporarios"

all_files = os.listdir(path_)

for file_ in all_files:
    
    file_split = file_.split(".")
    

    if file_split[1] == 'xlsx':
        date_ = datetime.strptime(file_split[0], '%Y-%m-%d')
        type_ = get_type(date_)

        file_path = path_ + file_
        
        if file_split[0] not in files_temp:
            if type_ == 1:
                process_type1(file_path, path_temp)
            elif type_ == 2:
                process_type2(file_path, path_temp)
            elif type_ == 3:
                process_type3(file_path, path_temp)
            elif type_ == 4:
                process_type4(file_path, path_temp)
            elif type_ == 5:
                process_type5(file_path, path_temp)
            elif type_ == 6:
                process_type6(file_path, path_temp)
        

    

### Processando os arquivos temporários

O código a seguir gera um arquivo único com os dados de todos os municípios ordenados por data e por nome do município. 

In [17]:
path_temp = "../tmp/boletins_temporarios"

all_files = os.listdir(path_temp)

all_df = []

for file_ in all_files:
    df_ = pd.read_csv(path_temp + "/" + file_)
    file_split = file_.split(".")
    df_['data'] = file_split[0]
    all_df.append(df_)

final_ = pd.concat(all_df, ignore_index=True)

final_ = final_.sort_values(by=['data','nome_municipio'])
final_ = final_[['data','codigo_municipio', 'nome_municipio', 'enviadas_dose1',
       'enviadas_dose2', 'enviadas_dose_unica', 'enviadas_dose_reforco',
       'aplicadas_dose1', 'aplicadas_dose2', 'aplicadas_dose_unica',
       'aplicadas_dose_reforco']]

final_ = final_.reset_index(drop=True)

final_

,data,codigo_municipio,nome_municipio,enviadas_dose1,enviadas_dose2,enviadas_dose_unica,enviadas_dose_reforco,aplicadas_dose1,aplicadas_dose2,aplicadas_dose_unica,aplicadas_dose_reforco
0,2021-02-03,2800100,Amparo de São Francisco,0,0,0,0,32,0,0,0
1,2021-02-03,2800209,Aquidabã,0,0,0,0,221,0,0,0
2,2021-02-03,2800308,Aracaju,0,0,0,0,8908,0,0,0
3,2021-02-03,2800407,Arauá,0,0,0,0,72,0,0,0
4,2021-02-03,2800506,Areia Branca,0,0,0,0,151,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
22420,2022-02-15,2807006,São Miguel do Aleixo,3578,3200,40,1284,3335,2725,54,788
22421,2022-02-15,2807303,Telha,2833,2393,45,979,2680,2144,46,846
22422,2022-02-15,2807402,Tobias Barreto,43656,34960,1160,9470,41207,35372,1187,12171
22423,2022-02-15,2807501,Tomar do Geru,11393,9725,135,4141,9205,7661,139,1961


In [18]:
final_.to_csv("../output/municipio_total.csv", index=False)

## Utilizando os dados produzidos 

Os trechos de códigos a seguir mostram como podemos obter o histórico de vacinação de um município específico ou de todo estado a partir da base produzida. 

In [19]:
data_final = pd.read_csv("../output/municipio_total.csv")
data_final

,data,codigo_municipio,nome_municipio,enviadas_dose1,enviadas_dose2,enviadas_dose_unica,enviadas_dose_reforco,aplicadas_dose1,aplicadas_dose2,aplicadas_dose_unica,aplicadas_dose_reforco
0,2021-02-03,2800100,Amparo de São Francisco,0,0,0,0,32,0,0,0
1,2021-02-03,2800209,Aquidabã,0,0,0,0,221,0,0,0
2,2021-02-03,2800308,Aracaju,0,0,0,0,8908,0,0,0
3,2021-02-03,2800407,Arauá,0,0,0,0,72,0,0,0
4,2021-02-03,2800506,Areia Branca,0,0,0,0,151,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
22420,2022-02-15,2807006,São Miguel do Aleixo,3578,3200,40,1284,3335,2725,54,788
22421,2022-02-15,2807303,Telha,2833,2393,45,979,2680,2144,46,846
22422,2022-02-15,2807402,Tobias Barreto,43656,34960,1160,9470,41207,35372,1187,12171
22423,2022-02-15,2807501,Tomar do Geru,11393,9725,135,4141,9205,7661,139,1961


### Total atual 

Como os dados na tabela são acumulativos a cada dia, para retornar o total atual seja do estado ou de um município, basta retornar os dados da última data disponível na base. 

Vejamos como seria para retornar o total de Sergipe.

In [20]:
total_estado = data_final[data_final['data'] == data_final.data.max()]
total_estado = total_estado.groupby(by=['data']).agg('sum')
total_estado = total_estado.reset_index(drop=False)
total_estado['codigo_municipio'] = 0
total_estado

,data,codigo_municipio,enviadas_dose1,enviadas_dose2,enviadas_dose_unica,enviadas_dose_reforco,aplicadas_dose1,aplicadas_dose2,aplicadas_dose_unica,aplicadas_dose_reforco
0,2022-02-15,0,2016868,1828087,39750,535047,1859371,1594038,40125,572623


Para o total de um município específico, basta filtrar os dados pelo município. 

In [21]:
total_municipio = data_final.query("codigo_municipio == 2800308")
total_municipio = total_municipio[total_municipio['data'] == total_municipio.data.max()]
total_municipio = total_municipio.groupby(by=['data']).agg('sum')
total_municipio = total_municipio.reset_index(drop=False)
total_municipio

,data,codigo_municipio,enviadas_dose1,enviadas_dose2,enviadas_dose_unica,enviadas_dose_reforco,aplicadas_dose1,aplicadas_dose2,aplicadas_dose_unica,aplicadas_dose_reforco
0,2022-02-15,2800308,603350,544967,12715,177355,533709,476443,12589,207422


### Dados históricos

Vamos agora agrupar os dados para ter um histórico da vacinação dia a dia, tanto do estado quanto do município. 

In [22]:
historico_estado = data_final.groupby(by=['data']).agg('sum')
historico_estado = historico_estado.reset_index()
historico_estado['codigo_municipio'] = 0
historico_estado

,data,codigo_municipio,enviadas_dose1,enviadas_dose2,enviadas_dose_unica,enviadas_dose_reforco,aplicadas_dose1,aplicadas_dose2,aplicadas_dose_unica,aplicadas_dose_reforco
0,2021-02-03,0,0,0,0,0,23577,0,0,0
1,2021-02-04,0,0,0,0,0,25415,0,0,0
2,2021-02-05,0,0,0,0,0,26393,0,0,0
3,2021-02-08,0,0,0,0,0,29086,0,0,0
4,2021-02-09,0,52284,23824,0,0,29517,273,0,0
...,...,...,...,...,...,...,...,...,...,...
294,2022-02-10,0,1977951,1806453,39750,520517,1838399,1585676,40125,536923
295,2022-02-11,0,2012291,1823001,39750,534717,1842894,1587457,40125,541921
296,2022-02-12,0,2012291,1823001,39750,534717,1846407,1589015,40125,550631
297,2022-02-14,0,2016868,1825975,39750,535047,1851812,1590920,40125,557958


Já para retornar o histórico por município basta fazer a `query` filtrando o município desejado.

In [23]:
historico_municipio = data_final.query("codigo_municipio == 2800308")
historico_municipio 

,data,codigo_municipio,nome_municipio,enviadas_dose1,enviadas_dose2,enviadas_dose_unica,enviadas_dose_reforco,aplicadas_dose1,aplicadas_dose2,aplicadas_dose_unica,aplicadas_dose_reforco
2,2021-02-03,2800308,Aracaju,0,0,0,0,8908,0,0,0
77,2021-02-04,2800308,Aracaju,0,0,0,0,9050,0,0,0
152,2021-02-05,2800308,Aracaju,0,0,0,0,9050,0,0,0
227,2021-02-08,2800308,Aracaju,0,0,0,0,11039,0,0,0
302,2021-02-09,2800308,Aracaju,23670,11169,0,0,11141,121,0,0
...,...,...,...,...,...,...,...,...,...,...,...
22052,2022-02-10,2800308,Aracaju,598250,536777,12715,177355,528411,474491,12589,194611
22127,2022-02-11,2800308,Aracaju,603350,544967,12715,177355,528411,474491,12589,194611
22202,2022-02-12,2800308,Aracaju,603350,544967,12715,177355,530833,475573,12589,201702
22277,2022-02-14,2800308,Aracaju,603350,544967,12715,177355,530833,475573,12589,201702
